In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima

path = '../data/processed_temperature_data.csv'

df = pd.read_csv(path, index_col=0)
T = df.values[:, :-2].flatten()[:-5]

plt.plot(T)
plt.show()

ModuleNotFoundError: No module named 'pmdarima'

In [4]:
# Automatically determine the ARIMA order
model = auto_arima(T, seasonal=True, stepwise=True, suppress_warnings=True)

# Forecast the next 120 timesteps
n_steps = 1200
forecast = model.predict(n_periods=n_steps)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(T, label='Historical Data')
plt.plot(np.arange(len(T), len(T) + n_steps), forecast, label='Forecast', color='red')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Forecast with auto_arima')
plt.legend()
plt.show()

NameError: name 'auto_arima' is not defined

In [99]:
# save data
F = forecast.reshape(-1, 12)
annual_mean = F.mean(axis=1).reshape(-1, 1)
annual_std = F.std(axis=1).reshape(-1, 1)
print('F shape', F.shape)
print('annual_mean shape', annual_mean.shape)
print('annual_std shape', annual_std.shape)

tmp = np.concatenate((F, annual_mean, annual_std), axis=1)

df_T_forecast_auto_arima = pd.DataFrame(tmp,)
df_T_forecast_auto_arima.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Annual', 'Annual_uncertainty']
df_T_forecast_auto_arima.index = pd.RangeIndex(start=2025, stop=2025+len(F))
df_T_forecast_auto_arima
df_T_forecast_auto_arima.to_csv('../data/T_forecast_auto_arima.csv')


F shape (100, 12)
annual_mean shape (100, 1)
annual_std shape (100, 1)
